In [1]:
import Pkg

Pkg.activate(@__DIR__)

In [3]:
using JuMP
using ToQUBO                 #px/indicator
using DisjunctiveProgramming #master
using DisjunctiveToQUBO
using QUBOTools
using LinearAlgebra

In [120]:
MOI.get(model::ToQUBO.Optimizer, ::MOI.TerminationStatus) = MOI.OPTIMAL

In [55]:
# Set parameters
cx = [1.5, 1]
cy = [6.1, 5.9]
α  = [0.75, 0.8]
d  = 3 # not specified in the problem, I think it is 3

3

$$
\begin{array}{rll}
    \displaystyle \min_{\mathbf{x}, \mathbf{Y}} & \mathbf{c_{x}}'\mathbf{x} + \mathbf{c_{y}}'\mathbf{y} \\
    \textrm{s.t.} & \mathbf{\alpha}'\mathbf{x} \ge d \\[0.75em]
                  & \displaystyle%
                    \bigvee_{i = 1}^{2} \left[\begin{array}{c} Y_{i} \\ x_{i} \leq 0 \end{array}\right] \\[1.75em]
                  & 0 \leq x_{i} \leq 5 & \forall i \\
                  & Y_{i} \iff y_{i} = 1 & \forall i \\
                  & Y_{i} \in \{\textrm{True}, \textrm{False}\} & \forall i \\
                  & y_{i} \in \{0, 1\} & \forall i \\[1em]
   \textrm{data}  & \mathbf{c_{x}} = [1.5, 1] \\
                  & \mathbf{c_{y}} = [6.1, 5.9] \\
                  & \mathbf{\alpha} = [0.75, 8] \\
                  & d = 3
\end{array}
$$

In [121]:
# Create the model
model = GDPModel(
    # Use ToQUBO in compilation mode
    () -> ToQUBO.Optimizer(nothing)
)

# Add the variables
@variable(model, 0 <= x[1:2] <= 5)
@variable(model, Y[1:2], Logical)

y = binary_variable.(Y)

set_attribute.(x, ToQUBO.Attributes.VariableEncodingMethod(), ToQUBO.Encoding.Arithmetic{Float64}())

# Add the objective
@objective(model, Min, cx' * x.^2 + cy' * y)

# Add the global constrainbt
@constraint(model, balance, α'x >= d)

# Create the disjunction constraint
@constraint(model, x[2] <= 0, Disjunct(Y[1]))
@constraint(model, x[1] <= 0, Disjunct(Y[2]))

disjunction(model, Y, exactly1 = true) # explcitly show the keyword to add an exactly 1 constraint 

# Compile the QUBO model
optimize!(model, gdp_method = Indicator())

Min 1.5 x[1]² + x[2]² + 6.1 y[1] + 5.9 y[2]
Subject to
 balance : 0.75 x[1] + 0.8 x[2] ≥ 3
 x[1] ≥ 0
 x[2] ≥ 0
 x[1] ≤ 5
 x[2] ≤ 5
 y[1] binary
 y[2] binary


In [113]:
n, ℓ, q, a, b = QUBOTools.qubo(unsafe_backend(model), :dense);

Q = q + diagm(ℓ)

display(Q)
println("Variables: $n")
println("Coefficient range: $(extrema(Q))")
println("Offset: $b")

21×21 Matrix{Float64}:
 -1.46091e5  67954.2         1.01931e5  …   302.0     0.0     0.0     0.0
  0.0           -2.58206e5   2.03862e5     -302.0     0.0     0.0     0.0
  0.0            0.0        -3.36343e5        0.0     0.0     0.0     0.0
  0.0            0.0         0.0              0.0   453.0   302.0     0.0
  0.0            0.0         0.0              0.0     0.0  -302.0   906.0
  0.0            0.0         0.0        …     0.0  -453.0     0.0  -906.0
  0.0            0.0         0.0              0.0  -453.0  -302.0  -906.0
  0.0            0.0         0.0           -302.0     0.0     0.0     0.0
  0.0            0.0         0.0              0.0     0.0     0.0     0.0
  0.0            0.0         0.0              0.0     0.0     0.0     0.0
  ⋮                                     ⋱                             ⋮
  0.0            0.0         0.0              0.0     0.0     0.0     0.0
  0.0            0.0         0.0              0.0     0.0     0.0     0.0
  0.0            

Variables: 21
Coefficient range: (-369648.0, 750847.5)
Offset: 391467.5
